In [1]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector as selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2025-03-17 19:01:25.944960: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 19:01:29.460321: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 19:01:31.574398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742238095.885649    2169 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742238096.640673    2169 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-17 19:01:43.578839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [21]:
import pandas as pd
df= pd.read_csv("./data/students_grading.csv")

In [22]:
df.head()

,Student_ID,First_Name,Last_Name,Email,Gender,Age,Department,Attendance (%),Midterm_Score,Final_Score,...,Projects_Score,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night
0,S1000,Omar,Williams,student0@university.com,Female,22,Engineering,52.29,55.03,57.82,...,85.90,56.09,F,6.2,No,Yes,High School,Medium,5,4.7
1,S1001,Maria,Brown,student1@university.com,Male,18,Engineering,97.27,97.23,45.80,...,55.65,50.64,A,19.0,No,Yes,NaN,Medium,4,9.0
2,S1002,Ahmed,Jones,student2@university.com,Male,24,Business,57.19,67.05,93.68,...,73.79,70.30,D,20.7,No,Yes,Master's,Low,6,6.2
3,S1003,Omar,Williams,student3@university.com,Female,24,Mathematics,95.15,47.79,80.63,...,92.12,61.63,A,24.8,Yes,Yes,High School,High,3,6.7
4,S1004,John,Smith,student4@university.com,Female,23,CS,54.18,46.59,78.89,...,68.42,66.13,F,15.4,Yes,Yes,High School,High,2,7.1


In [23]:
df.dtypes

Student_ID                     object
First_Name                     object
Last_Name                      object
Email                          object
Gender                         object
Age                             int64
Department                     object
Attendance (%)                float64
Midterm_Score                 float64
Final_Score                   float64
Assignments_Avg               float64
Quizzes_Avg                   float64
Participation_Score           float64
Projects_Score                float64
Total_Score                   float64
Grade                          object
Study_Hours_per_Week          float64
Extracurricular_Activities     object
Internet_Access_at_Home        object
Parent_Education_Level         object
Family_Income_Level            object
Stress_Level (1-10)             int64
Sleep_Hours_per_Night         float64
dtype: object

In [24]:
df.isna().sum()/df.shape[0]

Student_ID                    0.0000
First_Name                    0.0000
Last_Name                     0.0000
Email                         0.0000
Gender                        0.0000
Age                           0.0000
Department                    0.0000
Attendance (%)                0.1032
Midterm_Score                 0.0000
Final_Score                   0.0000
Assignments_Avg               0.1034
Quizzes_Avg                   0.0000
Participation_Score           0.0000
Projects_Score                0.0000
Total_Score                   0.0000
Grade                         0.0000
Study_Hours_per_Week          0.0000
Extracurricular_Activities    0.0000
Internet_Access_at_Home       0.0000
Parent_Education_Level        0.3588
Family_Income_Level           0.0000
Stress_Level (1-10)           0.0000
Sleep_Hours_per_Night         0.0000
dtype: float64

In [25]:
df.select_dtypes(include=['object']).nunique()

Student_ID                    5000
First_Name                       8
Last_Name                        6
Email                         5000
Gender                           2
Department                       4
Grade                            5
Extracurricular_Activities       2
Internet_Access_at_Home          2
Parent_Education_Level           4
Family_Income_Level              3
dtype: int64

In [26]:
df.select_dtypes(include=['object']).columns

Index(['Student_ID', 'First_Name', 'Last_Name', 'Email', 'Gender',
       'Department', 'Grade', 'Extracurricular_Activities',
       'Internet_Access_at_Home', 'Parent_Education_Level',
       'Family_Income_Level'],
      dtype='object')

In [27]:
for col in df.select_dtypes(include=['object']).columns:
    if df[col].nunique() < 10:
        print(f"{col:-<50}: {df[col].unique()}")

First_Name----------------------------------------: ['Omar' 'Maria' 'Ahmed' 'John' 'Liam' 'Sara' 'Emma' 'Ali']
Last_Name-----------------------------------------: ['Williams' 'Brown' 'Jones' 'Smith' 'Davis' 'Johnson']
Gender--------------------------------------------: ['Female' 'Male']
Department----------------------------------------: ['Engineering' 'Business' 'Mathematics' 'CS']
Grade---------------------------------------------: ['F' 'A' 'D' 'B' 'C']
Extracurricular_Activities------------------------: ['No' 'Yes']
Internet_Access_at_Home---------------------------: ['Yes' 'No']
Parent_Education_Level----------------------------: ['High School' nan "Master's" 'PhD' "Bachelor's"]
Family_Income_Level-------------------------------: ['Medium' 'Low' 'High']


In [28]:
# encode theTarget "Grade" column
df['Grade'] = df['Grade'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'F': 4})


In [29]:
# drop StudentID, Firstname, email , lastname 
X=df.drop(columns=["Student_ID", "First_Name", "Last_Name", "Email", "Grade"], axis=1)
y=df["Grade"]

In [30]:
# Split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [32]:
categorical_features= X.select_dtypes(exclude="number").columns
categorical_features

Index(['Gender', 'Department', 'Extracurricular_Activities',
       'Internet_Access_at_Home', 'Parent_Education_Level',
       'Family_Income_Level'],
      dtype='object')

In [33]:
numeric_features= X.select_dtypes("number").columns
numeric_features

Index(['Age', 'Attendance (%)', 'Midterm_Score', 'Final_Score',
       'Assignments_Avg', 'Quizzes_Avg', 'Participation_Score',
       'Projects_Score', 'Total_Score', 'Study_Hours_per_Week',
       'Stress_Level (1-10)', 'Sleep_Hours_per_Night'],
      dtype='object')

In [34]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True, drop='first'))])



In [35]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [36]:
# Applying the preprocessing pipeline
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [40]:
y_train.unique()

array([0, 1, 3, 4, 2])

In [43]:
# Neural network model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


# Neural network model
num_classes = len(np.unique(y_train))

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [44]:
# Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [45]:
# Training the model
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2019 - loss: 1.6691 - val_accuracy: 0.3040 - val_loss: 1.5576
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3296 - loss: 1.5321 - val_accuracy: 0.3720 - val_loss: 1.4115
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3639 - loss: 1.4319 - val_accuracy: 0.3650 - val_loss: 1.3730
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3725 - loss: 1.3864 - val_accuracy: 0.3620 - val_loss: 1.3501
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3613 - loss: 1.3812 - val_accuracy: 0.3560 - val_loss: 1.3443
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3810 - loss: 1.3687 - val_accuracy: 0.3630 - val_loss: 1.3325
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3751 - loss: 1.3385 - val_accuracy: 0.3720 - val_loss: 1.3222
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3751 - loss: 1.3356 - val_accuracy: 0.

In [46]:
# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3902 - loss: 1.2604  
Test Accuracy: 36.80%


In [50]:
# Make predictions (get the predicted classes)
y_pred_probs = model.predict(X_test)  # Get probabilities
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class predictions

 1/32 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [51]:
# Calculate metrics
from sklearn.metrics import classification_report, recall_score, f1_score

report = classification_report(y_test, y_pred)
recall_macro = recall_score(y_test, y_pred, average='macro')
f1_macro = f1_score(y_test, y_pred, average='macro')


In [52]:
# Print metrics

print("Classification Report:\n", report)
print("Recall (Macro):", recall_macro)
print("F1-score (Macro):", f1_macro)

Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.70      0.64       299
           1       0.20      0.01      0.01       195
           2       0.33      0.03      0.05       159
           3       0.34      0.42      0.37       178
           4       0.20      0.48      0.28       169

    accuracy                           0.37      1000
   macro avg       0.33      0.32      0.27      1000
weighted avg       0.36      0.37      0.31      1000

Recall (Macro): 0.324191577930541
F1-score (Macro): 0.2696788139588537
